In [1]:
from hmmlearn import hmm
from abc_utils import *
import numpy as np
from tqdm import tqdm
RANDOM_STATE = 42

In [2]:
train_set, _ = load_harmonization_train_test()

In [3]:
abc_texts = train_set.sample(200, random_state=RANDOM_STATE)['output']

In [4]:
# list_o_dfs = [abc_to_dataframe(abc_text) for abc_text in tqdm(abc_texts)]

In [5]:
l_o_df = []
l_o_df_lengths = []
for abc_text in tqdm(abc_texts):
    try:
        df = abc_to_dataframe(abc_text)
    except Exception as e:
        print("bugged out", e)
    lengths = len(df)
    l_o_df.append(df)
    l_o_df_lengths.append(lengths)

  0%|          | 0/200 [00:00<?, ?it/s]

 68%|██████▊   | 136/200 [00:31<00:13,  4.67it/s]

bugged out Cannot make a step out of '^'


 76%|███████▌  | 152/200 [00:35<00:13,  3.69it/s]

bugged out Cannot make a step out of '^'


 78%|███████▊  | 156/200 [00:36<00:08,  5.37it/s]

bugged out 'Fr'


 98%|█████████▊| 197/200 [00:43<00:00,  6.79it/s]

bugged out '#vii'


100%|██████████| 200/200 [00:43<00:00,  4.55it/s]


In [ ]:
music_df = pd.concat(l_o_df)
# music_df.replace('-', 0, inplace=True)
print(music_df['melody'].value_counts())
print(music_df.__len__())
state_chords = music_df.chord.values
obs_melody = music_df.melody.values
symbols = sorted(list(set(obs_melody)))
print(len(symbols), symbols)


melody
79    1879
77    1859
72    1643
82    1381
75    1300
81    1130
74    1042
84    1007
80     991
0      912
76     817
78     759
70     744
73     718
86     619
83     554
87     451
69     420
68     403
67     362
65     345
85     302
71     284
89     215
91     176
88     163
90     124
66     107
64      89
63      39
93      35
60      31
92      31
62      17
61       8
95       4
94       3
59       1
Name: count, dtype: int64
20965
38 [0, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]


In [10]:
h = hmm.CategoricalHMM(n_components=5, n_iter=1000, tol=1e-6)
h.fit(obs_melody.reshape(-1, 1), lengths=l_o_df_lengths)
pi = h.startprob_
A = h.transmat_.T
B = h.emissionprob_.T

In [11]:
B.shape

(96, 5)

In [13]:
np.sum(h.emissionprob_.T, axis=0)

array([1., 1., 1., 1., 1.])

In [16]:
for i in range(len(B)):
    if i in symbols:
        print(u"{}, {:0.4f}, {:0.4f}, {:0.4f}, {:0.4f}, {:0.4f}".format(i, *B[i,:]))

0, 0.0101, 0.0013, 0.0191, 0.1688, 0.1936
59, 0.0000, 0.0000, 0.0000, 0.0004, 0.0000
60, 0.0000, 0.0000, 0.0000, 0.0003, 0.0168
61, 0.0000, 0.0000, 0.0000, 0.0035, 0.0000
62, 0.0000, 0.0000, 0.0000, 0.0075, 0.0000
63, 0.0000, 0.0000, 0.0000, 0.0070, 0.0128
64, 0.0000, 0.0000, 0.0000, 0.0154, 0.0299
65, 0.0000, 0.0000, 0.0000, 0.1147, 0.0469
66, 0.0000, 0.0000, 0.0001, 0.0161, 0.0386
67, 0.0000, 0.0019, 0.0001, 0.0281, 0.1586
68, 0.0000, 0.0000, 0.0002, 0.1429, 0.0426
69, 0.0000, 0.0000, 0.0000, 0.1075, 0.0976
70, 0.0000, 0.0464, 0.0000, 0.0615, 0.1782
71, 0.0000, 0.0144, 0.0000, 0.0329, 0.0675
72, 0.0000, 0.1468, 0.0049, 0.2623, 0.0648
73, 0.0000, 0.1016, 0.0000, 0.0179, 0.0316
74, 0.0005, 0.1591, 0.0032, 0.0051, 0.0199
75, 0.0019, 0.1752, 0.0322, 0.0000, 0.0000
76, 0.0032, 0.1079, 0.0198, 0.0037, 0.0000
77, 0.0000, 0.1830, 0.1065, 0.0016, 0.0000
78, 0.0024, 0.0350, 0.0773, 0.0000, 0.0000
79, 0.0144, 0.0253, 0.2399, 0.0018, 0.0000
80, 0.0122, 0.0017, 0.1346, 0.0000, 0.0000
81, 0.0409, 